---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df = pd.DataFrame(df.apply(lambda x:x.rstrip()),columns=['text'])

In [2]:
# Extract relevant data
year_only = df['text'].str.findall(r'\b[a-z]?[12][09][0-9][0-9]\b') # Year only
nums_dmy2 = df['text'].str.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2}\b') # xx/xx/xx
nums_dmy4 = df['text'].str.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{4}\b') # xx/xx/xxxx
nums_my = df['text'].str.findall(r'\d{1,2}[/]\d{4}\b')
num_Jan_Year = df['text'].str.findall(r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December)\
[/-]?[,]?[.]? \d{4}') # xx/Jan/yyyy
Jan_num_Year = df['text'].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December)\
[/-]?[,]?[.]? \d{1,2}[a-z]*[,]?[.]? \d{4}') # xx/Jan/yyyy
Jan_Year = df['text'].str.findall(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December)\
[/-]?[,]?[.]? \d{4}\b') # Jan/Year


In [3]:
# Re-arrange relevant data
import numpy as np
YMD = pd.DataFrame(np.vstack([year_only,nums_dmy2,nums_dmy4,nums_my,num_Jan_Year,Jan_num_Year,Jan_Year])).T
YMD.rename(columns ={0:'Year_only',1:'mm/dd/yy',2:'mm/dd/yyyy',3:'mm/yyyy',4:'dd/Jan/Year',5:'Jan/dd/Year',6:'Jan/Year'},inplace = True)

YMD = pd.DataFrame(YMD[C].apply(lambda x:'N' if len(x) ==0 else x) for C in YMD.columns).T

# convert list to string
for col in YMD.columns:
    for i in np.arange(len(YMD)):
        try:
            YMD[col][i] = YMD[col][i][0]
        except: YMD[col][i] = YMD[col][i]



In [4]:
# map month to numeric values, create dictionary
import calendar
mon_to_num = {v:k for k,v in enumerate(calendar.month_abbr) if v}
month_to_num = {v:k for k,v in enumerate(calendar.month_name) if v}
mon_to_num.update(month_to_num)

a = YMD['Jan/dd/Year'][YMD['Jan/dd/Year'] !='N'].str.split(' ').apply(lambda x:x[0]).apply(lambda x:x.rstrip(r'[.-]')).map(mon_to_num)
b = YMD['dd/Jan/Year'][YMD['dd/Jan/Year'] !='N'].str.split(' ').apply(lambda x:x[1]).apply(lambda x:x.rstrip(r'[.,-]')).map(mon_to_num)
c = YMD['Jan/Year'][YMD['Jan/Year'] !='N'].str.split(' ').apply(lambda x:x[0]).apply(lambda x:x.rstrip(r'[.,-]')).map(mon_to_num)

In [5]:
# rearrange the format of all days to mm/dd/yy
from datetime import datetime


YMD['Jan/dd/Year'][YMD['Jan/dd/Year'] !='N'] = YMD['Jan/dd/Year'][YMD['Jan/dd/Year'] !='N'].str.split(' ')
YMD['dd/Jan/Year'][YMD['dd/Jan/Year'] !='N'] = YMD['dd/Jan/Year'][YMD['dd/Jan/Year'] !='N'].str.split(' ')
YMD['Jan/Year'][YMD['Jan/Year'] !='N'] = YMD['Jan/Year'][YMD['Jan/Year'] !='N'].str.split(' ')
YMD['Year_only'][YMD['Year_only']!='N'] = YMD['Year_only'][YMD['Year_only'] !='N'].apply(lambda x:x.strip(r'[ry]')) # why [a-z] not working?
# might use datetime.strptime() and datetime.strftime() instead
YMD['Year_only'][YMD['Year_only']!='N'] = YMD['Year_only'][YMD['Year_only'] !='N'].apply(lambda x:x[-2:]) # because only yy not yyyy


for i in YMD['Jan/dd/Year'][YMD['Jan/dd/Year'] !='N'].index:
    YMD['Jan/dd/Year'][i][0] = a[i]
    YMD['Jan/dd/Year'][i][1] =  YMD['Jan/dd/Year'][i][1].rstrip(',')
    YMD['Jan/dd/Year'][i] = '{}/{}/{}'.format(str(YMD['Jan/dd/Year'][i][0]),str(YMD['Jan/dd/Year'][i][1]),str(YMD['Jan/dd/Year'][i][2][-2:]))

for i in YMD['dd/Jan/Year'][YMD['dd/Jan/Year'] !='N'].index:
    YMD['dd/Jan/Year'][i][1] = b[i]
    YMD['dd/Jan/Year'][i] = '{}/{}/{}'.format(str(YMD['dd/Jan/Year'][i][1]),str(YMD['dd/Jan/Year'][i][0]),str(YMD['dd/Jan/Year'][i][2][-2:]))
    
for i in YMD['Jan/Year'][YMD['Jan/Year'] !='N'].index:
    YMD['Jan/Year'][i][0] = c[i]
    YMD['Jan/Year'][i] = '{}/1/{}'.format(str(YMD['Jan/Year'][i][0]),str(YMD['Jan/Year'][i][1][-2:]))

for i in YMD['mm/dd/yyyy'][YMD['mm/dd/yyyy']!='N'].index:
    YMD['mm/dd/yyyy'][i] = datetime.strftime(datetime.strptime(YMD['mm/dd/yyyy'][i],'%m/%d/%Y'),'%m/%d/%y') # Zero-padding not working !

for i in YMD['mm/yyyy'][YMD['mm/yyyy']!='N'].index:
    YMD['mm/yyyy'][i] = datetime.strftime(datetime.strptime(YMD['mm/yyyy'][i],'%d/%Y'),'%d/%y') # Because overlap with mm/dd/yyyy and bad RE
    YMD['mm/yyyy'][i] = '{}/1/{}'.format(YMD['mm/yyyy'][i].split('/')[0],YMD['mm/yyyy'][i].split('/')[1])

In [6]:
# Nan 'N's
YMD = pd.DataFrame(YMD[C].apply(lambda x:np.nan if len(x) ==1 else x) for C in YMD.columns).T

# new column to store standard date
YMD['YMD'] = YMD['mm/dd/yy'] 

# fillna

YMD['YMD'].fillna(YMD['mm/dd/yyyy'],inplace = True)
YMD['YMD'].fillna(YMD['mm/yyyy'],inplace = True) # have been re-formed
YMD['YMD'].fillna(YMD['Jan/dd/Year'],inplace = True)
YMD['YMD'].fillna(YMD['dd/Jan/Year'],inplace = True)
YMD['YMD'].fillna(YMD['Jan/Year'],inplace = True)
YMD['YMD'].fillna('1/1/'+YMD['Year_only'],inplace = True) # re-form it

In [7]:
# standard format
for i in np.arange(len(YMD)):
    YMD.YMD[i] = '/'.join(YMD.YMD[i].split('-'))

# remove all '0's, why '%-d' or '%#d' not working?
day_no_padding = YMD.YMD.str.split('/').apply(lambda x:x[0].lstrip('0'))
mon_no_padding = YMD.YMD.str.split('/').apply(lambda x:x[1].lstrip('0'))
year_no_padding = YMD.YMD.str.split('/').apply(lambda x:x[2])

for i in np.arange(len(YMD)):
    YMD.YMD[i] = '{}/{}/{}'.format(day_no_padding[i],mon_no_padding[i],year_no_padding[i])

In [8]:
# sort date
indices = pd.Series([indices for indices,values in sorted(enumerate(YMD.YMD),key=lambda x:datetime.strptime(x[-1],'%m/%d/%y'))])# meaning to sort dates with such format

In [9]:
def date_sorter():
    
    # Your code here
    
    return indices
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    235
489    475
490    257
491    152
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64